In [1]:
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional, MaxPooling1D, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform, he_uniform, zeros
from keras.optimizers import SGD, Adam, Adadelta
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras import metrics
from sklearn.externals import joblib

C:\Users\mehak\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def import_textFile(filePath):
    ds_text = []
    with open(filePath,'rb') as f:
        for idx,ln in enumerate(f):
            decoded=False
            line=''
            for cp in ('cp1252', 'cp850','utf-8','utf8'):
                try:
                    line = ln.decode(cp)
                    decoded=True
                    break
                except UnicodeDecodeError:
                    pass
            if decoded:
                ds_text.insert(idx, line.rstrip())
    return ds_text

In [3]:
emo=5

In [4]:
if emo == 2: 
    dataset_path = '../Data/Datasets/Binary Classification/'
else:
    dataset_path = '../Data/Datasets/Multiclass Classification/'
    
train_ds = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test_ds = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)

x_train = train_ds[0]
y_train = train_ds[1]

x_test = test_ds[0]
y_test = test_ds[1]

In [5]:
maxLen = len(max(x_train, key=len).split())
print(maxLen)

28


In [6]:
def get_one_hot(y):
    one_hot_temp_array = array(y)
    one_hot_array = to_categorical(one_hot_temp_array)
    return one_hot_array
# invert encoding
#inverted = argmax(encoded[0])

In [7]:
#load glove diembedding_dict = dict()
embedding_dict = joblib.load('../Data/rnn_embedding_dict.pkl')
glove_words = joblib.load('../Data/rnn_glove_words.pkl')
number_to_word = glove_words
word_to_number = dict((word,idx) for idx,word in enumerate(glove_words))

In [8]:
def get_indices(input_x, word_to_number, max_len):
    m = input_x.shape[0]
    x_indices = np.zeros((m,max_len))

    for i in range(m):
        word_list =input_x.iloc[i][0].lower().split()
        for idx,word in enumerate(word_list):
            if (idx ==32):
                continue
            x_indices[i, idx] = word_to_number[word]
    return x_indices

In [9]:
def get_embedding_layer(embedding_dict, word_to_number):
    
    emb_shape = embedding_dict["cucumber"].shape[0]
    total_words = len(word_to_number) + 1
    
    emb_matrix = np.zeros((total_words,emb_shape))
    
    for word,idx in word_to_number.items():
        emb_matrix[idx, :] = embedding_dict[word]

    embedding_layer = Embedding(total_words, emb_shape, trainable = True)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [10]:
def senti_model(input_shape,embedding_dict, word_to_number):
    #np.random.seed(1)
    sentence_indices = Input(shape = input_shape, dtype = np.int32)
    embedding_layer = get_embedding_layer(embedding_dict, word_to_number)
    
    embeddings = embedding_layer(sentence_indices)   
    X = Dropout(0.4)(embeddings)
    X = Bidirectional(GRU(64,return_sequences=True))(X)
    X = Dense(16, activation='elu')(X)
    X = Bidirectional(GRU(64,return_sequences=False))(X)
    X = Dense(64, activation='elu')(X)
    X = Dense(emo, activation='softmax', name='fc')(X)



    #X = Activation('softmax')(X)
    
    model = Model(inputs = sentence_indices, outputs = X, name='sentiment')
    
    return model

In [11]:
model = senti_model((maxLen,), embedding_dict, word_to_number)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 28, 50)            20000000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 28, 128)           44160     
_________________________________________________________________
dense_1 (Dense)              (None, 28, 16)            2064      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               31104     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
__________

In [12]:
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
adam = Adam(lr=0.0001)
adadelta = Adadelta(lr=1.00, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adadelta, metrics=[metrics.categorical_accuracy])

In [ ]:
x_train_indices = get_indices(x_train, word_to_number, maxLen)
y_train_oh = get_one_hot(y_train)
print(y_train_oh.shape)

(2809, 5)


In [ ]:
model.fit(x_train_indices, y_train_oh, epochs = 200, batch_size = 64, shuffle=True)

Epoch 1/200
2809/2809 [==============================] - 28s 10ms/step - loss: 1.5328 - categorical_accuracy: 0.2723
Epoch 2/200
2809/2809 [==============================] - 21s 8ms/step - loss: 1.5221 - categorical_accuracy: 0.2919
Epoch 3/200
2809/2809 [==============================] - 22s 8ms/step - loss: 1.5202 - categorical_accuracy: 0.2880
Epoch 4/200
2809/2809 [==============================] - 23s 8ms/step - loss: 1.5202 - categorical_accuracy: 0.2955
Epoch 5/200
2809/2809 [==============================] - 23s 8ms/step - loss: 1.5142 - categorical_accuracy: 0.2955
Epoch 6/200
2809/2809 [==============================] - 23s 8ms/step - loss: 1.5183 - categorical_accuracy: 0.2919
Epoch 7/200
2809/2809 [==============================] - 24s 9ms/step - loss: 1.5148 - categorical_accuracy: 0.3008
Epoch 8/200
2809/2809 [==============================] - 24s 9ms/step - loss: 1.5155 - categorical_accuracy: 0.2983
Epoch 9/200
2809/2809 [==============================] - 25s 9ms/step -

2809/2809 [==============================] - 45s 16ms/step - loss: 1.5009 - categorical_accuracy: 0.3072
Epoch 71/200
2809/2809 [==============================] - 48s 17ms/step - loss: 1.5006 - categorical_accuracy: 0.3111
Epoch 72/200
2809/2809 [==============================] - 48s 17ms/step - loss: 1.5025 - categorical_accuracy: 0.3079
Epoch 73/200
2809/2809 [==============================] - 49s 18ms/step - loss: 1.5007 - categorical_accuracy: 0.3186
Epoch 74/200
2809/2809 [==============================] - 49s 17ms/step - loss: 1.4997 - categorical_accuracy: 0.3133
Epoch 75/200
2809/2809 [==============================] - 44s 16ms/step - loss: 1.4983 - categorical_accuracy: 0.3108
Epoch 76/200
2809/2809 [==============================] - 35s 13ms/step - loss: 1.4991 - categorical_accuracy: 0.3176
Epoch 77/200
2809/2809 [==============================] - 35s 12ms/step - loss: 1.5013 - categorical_accuracy: 0.3126
Epoch 78/200
2809/2809 [==============================] - 34s 12ms/st

2809/2809 [==============================] - 23s 8ms/step - loss: 1.4955 - categorical_accuracy: 0.3133
Epoch 140/200
2809/2809 [==============================] - 26s 9ms/step - loss: 1.4966 - categorical_accuracy: 0.3122
Epoch 141/200
2809/2809 [==============================] - 24s 9ms/step - loss: 1.4981 - categorical_accuracy: 0.3161
Epoch 142/200
2809/2809 [==============================] - 23s 8ms/step - loss: 1.4962 - categorical_accuracy: 0.3208
Epoch 143/200
2809/2809 [==============================] - 23s 8ms/step - loss: 1.4980 - categorical_accuracy: 0.3136
Epoch 144/200
2809/2809 [==============================] - 23s 8ms/step - loss: 1.4958 - categorical_accuracy: 0.3154
Epoch 145/200
2809/2809 [==============================] - 23s 8ms/step - loss: 1.4975 - categorical_accuracy: 0.3151
Epoch 146/200
2809/2809 [==============================] - 24s 9ms/step - loss: 1.4970 - categorical_accuracy: 0.3094
Epoch 147/200
2809/2809 [==============================] - 24s 8ms/ste

In [ ]:
x_test_indices = get_indices(x_test, word_to_number, max_len = maxLen)
y_test_oh = get_one_hot(y_test)
loss, acc = model.evaluate(x_test_indices, y_test_oh)
print()
print("Test accuracy = ", acc)